Ok - let's try to run scoary.

First, I installed using `pip install scoary` - no issues


Now let's pull over trish's csv

In [6]:
%cd ~/scoary
!rclone sync dbox:/timplab_data/cpowgs/201104_scoary/ ~/scoary/
!rclone copy dbox:/timplab_data/cpowgs/201103_tree/klpn.csv ~/scoary/

/home/timp/scoary


In [69]:
import pandas as pd
import numpy as np

Note - the keep_default_na False is really important for keeping empty strings empty, IMO.

In [124]:
fullmeta=pd.read_csv("~/scoary/klpn.csv", keep_default_na=False)
pheno=pd.read_csv("~/scoary/from_trish_edited.csv", keep_default_na=False)
fullmeta.replace(np.nan, '', regex=True)


,Unnamed: 0,id,cp,cp_gene,mlst,repA,year,pilon,spades,illR1,illR2
0,0,6,CP,KPC-3,ST-16,Yes,2016,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
1,1,9,CP,KPC-2,ST-258,,2016,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
2,2,16,Non-CP,,ST-268,,2016,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
3,3,19,CP,KPC-3,ST-11,Yes,2016,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
4,4,21,Non-CP,,ST-17,,2016,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
...,...,...,...,...,...,...,...,...,...,...,...
116,116,640,CP,KPC-2,ST-258,Yes,2018,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
117,117,643,Non-CP,,ST-15,,2018,,klpn_spades/643/scaffolds.fasta,/atium/Data/Nanopore/cpowgs/illumina_4/643KLPN...,/atium/Data/Nanopore/cpowgs/illumina_4/643KLPN...
118,118,645,Non-CP,,ST-14,,2018,,klpn_spades/645/scaffolds.fasta,/atium/Data/Nanopore/cpowgs/illumina_4/645KLPN...,/atium/Data/Nanopore/cpowgs/illumina_4/645KLPN...
119,119,647,CP,KPC-3,ST-258,Yes,2018,/atium/Data/Nanopore/cpowgs/assemblies/pilon/K...,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...


Make assembly label (I really should have done this in original script)

In [127]:
fullmeta["label"]=""
for index, sample in fullmeta.iterrows():
    if sample.pilon:
        fullmeta.loc[index, 'label']="klpn_pilon_"+str(sample.id)
    else:
        if sample.spades:
            fullmeta.loc[index, 'label']="klpn_spades_"+str(sample.id)


Do a join

In [130]:
phenmeta=pd.merge(fullmeta, pheno, on="id", how="left")
phenmeta=phenmeta[phenmeta.cef.notna()]

Set the trait to 0 or 1 as requested by scoary

In [131]:
phenmeta['cef_trait']=np.where(phenmeta.cef=='yes', 1, 0)

get just relevant columns and output to csv

In [132]:
outphen=phenmeta[['cef_trait', 'label']]

In [133]:
outphen=outphen.set_index('label')

In [134]:
outphen.index.name=''

In [135]:
outphen.to_csv("~/scoary/pheno_proc.csv")

Ok - with processed csv in hand, run scoary:

In [139]:
!scoary -g /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree/pan_klpn/gene_presence_absence.csv -t ~/scoary/pheno_proc.csv

==== Scoary started ====
Command: /home/timp/miniconda3/envs/cpowgs/bin/scoary -g /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree/pan_klpn/gene_presence_absence.csv -t /home/timp/scoary/pheno_proc.csv
Reading gene presence absence file
Creating Hamming distance matrix based on gene presence/absence
Building UPGMA tree from distance matrix
Reading traits file
ERROR: Some isolates in your gene presence absence file were not represented in your traits file. These will count as MISSING data and will not be included.
Finished loading files into memory.


==== Performing statistics ====
-- Filtration options --
Individual (Naive):    0.05
Collapse genes:    False


Tallying genes and performing statistical analyses
Gene-wise counting and Fisher's exact tests for trait: cef_trait
100.00%
Adding p-values adjusted for testing multiple hypotheses

Storing results: cef_trait
Calculating max number of contrasting pairs for each nominally significant gene
100.00%
Storing results to file


==== F

In [140]:
!rclone sync ~/scoary/ dbox:/timplab_data/cpowgs/201104_scoary/
